In [2]:
import pandas as pd

df = pd.read_csv('main.csv')

# Data Quality Assessment

In [3]:
# create new dataframe with relevant columns
cols = ['route_id','status','cancellation_reason','decline_reason','operator_name']
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   route_id             5000 non-null   object
 1   status               5000 non-null   object
 2   cancellation_reason  3557 non-null   object
 3   decline_reason       1531 non-null   object
 4   operator_name        5000 non-null   object
dtypes: object(5)
memory usage: 195.4+ KB


In [4]:
df.sample(5)

,route_id,status,cancellation_reason,decline_reason,operator_name
2308,guatemala-city-to-antigua,canceled,NaN,other,GuateGo
1829,split-to-hvar,canceled,"online cancel - Booking error (wrong date, dep...",NaN,Krilo (Kapetan Luka)
1936,foz-do-iguacu-to-puerto-iguazu,canceled,NaN,other,Caracol Internacional
2850,calama-to-uyuni,canceled,online cancel - My plans changed,NaN,Cruz del Norte
1983,ha-tien-to-ho-chi-minh-city-saigon,canceled,"online cancel - Booking error (wrong date, dep...",NaN,Futa Bus


In [5]:
# describe all columns
df.describe(include='all')

,route_id,status,cancellation_reason,decline_reason,operator_name
count,5000,5000,3557,1531,5000
unique,1168,4,317,13,586
top,split-to-hvar,canceled,online cancel - My plans changed,fully_booked,Krilo (Kapetan Luka)
freq,120,4509,1503,638,403


In [6]:
# Explore missing values for relevant columns
df.isnull().sum()

route_id                  0
status                    0
cancellation_reason    1443
decline_reason         3469
operator_name             0
dtype: int64

In [8]:

# filter where decline_reason is not nan
df_canceled = df[df['status'] == 'canceled']
print(df_canceled.isnull().sum())
print(f'Number of canceled applications: {df_canceled.shape[0]}')
# count when declined_reason is not na
print(f'Number of canceled applications with decline reason: {df_canceled["decline_reason"].count()}')
# count when cancelled_reason is not na
print(f'Number of canceled applications with cancel reason: {df_canceled["cancellation_reason"].count()}')
# explore decline_reason for canceled applications
df_canceled['decline_reason'].value_counts()

route_id                  0
status                    0
cancellation_reason    1084
decline_reason         3337
operator_name             0
dtype: int64
Number of canceled applications: 4509
Number of canceled applications with decline reason: 1172
Number of canceled applications with cancel reason: 3425


decline_reason
fully_booked                     584
other                            152
pickup_location_not_supported     96
departure_does_not_exists         79
departure_cancelled               56
rate_difference                   41
missing_info_from_client          37
not_approved_on_time              36
not_enough_passengers             28
schedule_is_not_open_yet          26
supplier_unresponsive             18
out_of_credit                     15
double_booking                     4
Name: count, dtype: int64

In [42]:
# count rows where both cancellation_reason and decline_reason are not na
df_both_specified = df[(df['cancellation_reason'].notna()) & (df['decline_reason'].notna())]
print(f'Number of rows where cancellation_reason and decline_reason are not na: {df_both_specified.shape[0]}')
df_both_specified.value_counts('status')

Number of rows where cancellation_reason and decline_reason are not na: 88


status
canceled    88
Name: count, dtype: int64

Inconsistencies in bookings where status is canceled
* 1172 bookings where status is cancelled 
* 88 bookings where cancellation reason and decline reason are both specified. 
* All of those have canceled status

In [86]:
df_approved = df[df['status'] == 'approved']
df_approved.shape[0]
df_approved.value_counts('decline_reason')
df_approved.value_counts('cancellation_reason')

print('Number of approved applications: ', df_approved.shape[0])
print('NUmber of approved rows where the cancellation_reason is not null: ', df_approved[df_approved['cancellation_reason'].notnull()].shape[0])
print('NUmber of approved rows where the decline_reason is not null: ', df_approved[df_approved['decline_reason'].notnull()].shape[0])

Number of approved applications:  468
NUmber of approved rows where the cancellation_reason is not null:  132
NUmber of approved rows where the decline_reason is not null:  336


Inconsistencies in bookings where status is canceled
* 468 bookings where status is approved 
* 132 of these  have cancellation reason
* 336 of these have decline_reason
* All of the approved bookings have cancellation reason or decline reason

In [76]:
df_approved.value_counts('decline_reason')

decline_reason
pickup_location_not_supported    77
missing_info_from_client         71
fully_booked                     46
other                            40
rate_difference                  40
departure_does_not_exists        38
departure_cancelled              15
schedule_is_not_open_yet          4
not_enough_passengers             2
double_booking                    1
not_approved_on_time              1
out_of_credit                     1
Name: count, dtype: int64

In [84]:
df_declined = df[df['status'] == 'declined']
# count cancellation_reasons not null
print(f"Declined applications where \
cancellation_reason is not null \
{df_declined['cancellation_reason'].count()}")

Declined applications where cancellation_reason is not null 0


*  Declined applications seem to be consistent